In [1]:
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 23),
 datetime.date(2020, 10, 22),
 datetime.date(2020, 10, 21))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201023.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201022.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201021.csv')

In [4]:
if False:
    while True:
        status_code = requests.get(url_tday).status_code
        if status_code == 200:
            print('Presenti dati odierni')
            break
        print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)

In [5]:
# Get csv files
try:
    # today and yesterday if today present
    df_raw_r_1 = pd.read_csv(url_tday)
    print('Dati odierni')
    df_raw_r_0 = pd.read_csv(url_yday)
    date_last = date_tday
except Exception as e:
    # yesterday and day before yesterday otherwise
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)
    date_last = date_yday
    print('Dati di ieri')

Dati odierni


In [6]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-23T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1362,84,1446,15016,...,1682,2032,31002,4236,27151,24549,51700,919629,547164,NaN
1,2020-10-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,50,3,53,948,...,96,97,1166,149,2076,240,2316,37111,23862,NaN
2,2020-10-23T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2013,184,2197,35753,...,4432,4916,88536,17159,119459,24186,143645,2647681,1655276,NaN
3,2020-10-23T17:00:00,ITA,5,Veneto,45.434905,12.338452,515,64,579,13455,...,1426,1550,24798,2308,23734,17406,41140,2208831,860733,NaN
4,2020-10-23T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,88,19,107,2370,...,252,340,4569,369,6792,623,7415,497736,216003,NaN
5,2020-10-23T17:00:00,ITA,7,Liguria,44.411493,8.932699,598,33,631,4838,...,403,778,14210,1680,15062,6297,21359,399461,209220,NaN
6,2020-10-23T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,657,86,743,11771,...,851,888,27304,4547,35818,8547,44365,1449195,790989,NaN
7,2020-10-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,574,82,656,13718,...,1113,1290,12290,1237,21997,5904,27901,973655,653719,Sono stati effettuati 1726 tamponi antigenici ...
8,2020-10-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,183,21,204,3426,...,360,447,2577,100,2287,4020,6307,269993,158186,NaN
9,2020-10-23T17:00:00,ITA,11,Marche,43.616760,13.518875,152,22,174,2875,...,425,453,6596,1000,10645,0,10645,297179,173274,NaN


In [7]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-22T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1226,79,1305,13475,...,1345,1550,30661,4227,26564,23104,49668,906964,537462,NaN
1,2020-10-22T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,48,3,51,854,...,74,87,1165,149,1988,231,2219,36671,23688,NaN
2,2020-10-22T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1685,156,1841,31677,...,3628,4125,88059,17152,116243,22486,138729,2610718,1633109,NaN
3,2020-10-22T17:00:00,ITA,5,Veneto,45.434905,12.338452,500,59,559,12049,...,1175,1325,24681,2301,23515,16075,39590,2192554,854703,NaN
4,2020-10-22T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,83,14,97,2128,...,119,220,4482,368,6490,585,7075,492435,214444,NaN
5,2020-10-22T17:00:00,ITA,7,Liguria,44.411493,8.932699,540,34,574,4492,...,445,690,13842,1673,14535,6046,20581,394365,207034,NaN
6,2020-10-22T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,636,86,722,10941,...,863,889,27277,4537,35402,8075,43477,1433217,783681,NaN
7,2020-10-22T17:00:00,ITA,9,Toscana,43.769231,11.255889,524,82,606,12655,...,1022,1145,12121,1229,20938,5673,26611,962277,646277,Sono stati effettuati 758 tamponi antigenici r...
8,2020-10-22T17:00:00,ITA,10,Umbria,43.106758,12.388247,171,22,193,3077,...,315,407,2493,97,2240,3620,5860,266398,156285,NaN
9,2020-10-22T17:00:00,ITA,11,Marche,43.616760,13.518875,133,19,152,2472,...,247,321,6570,998,10192,0,10192,292862,171050,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-23T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1362,84,1446,15016,...,1682,2032,31002,4236,27151,24549,51700,919629,547164,NaN
1,2020-10-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,50,3,53,948,...,96,97,1166,149,2076,240,2316,37111,23862,NaN
2,2020-10-23T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2013,184,2197,35753,...,4432,4916,88536,17159,119459,24186,143645,2647681,1655276,NaN
3,2020-10-23T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,171,11,182,3529,...,424,476,8914,719,10098,3246,13344,484932,221317,NaN
4,2020-10-23T17:00:00,ITA,5,Veneto,45.434905,12.338452,515,64,579,13455,...,1426,1550,24798,2308,23734,17406,41140,2208831,860733,NaN
5,2020-10-23T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,88,19,107,2370,...,252,340,4569,369,6792,623,7415,497736,216003,NaN
6,2020-10-23T17:00:00,ITA,7,Liguria,44.411493,8.932699,598,33,631,4838,...,403,778,14210,1680,15062,6297,21359,399461,209220,NaN
7,2020-10-23T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,657,86,743,11771,...,851,888,27304,4547,35818,8547,44365,1449195,790989,NaN
8,2020-10-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,574,82,656,13718,...,1113,1290,12290,1237,21997,5904,27901,973655,653719,Sono stati effettuati 1726 tamponi antigenici ...
9,2020-10-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,183,21,204,3426,...,360,447,2577,100,2287,4020,6307,269993,158186,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-22T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1226,79,1305,...,1345,1550,30661,4227,26564,23104,49668,906964,537462,NaN
1,17,2020-10-22T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,48,3,51,...,74,87,1165,149,1988,231,2219,36671,23688,NaN
2,8,2020-10-22T17:00:00,ITA,3,Lombardia,45.466794,9.190347,1685,156,1841,...,3628,4125,88059,17152,116243,22486,138729,2610718,1633109,NaN
3,19,2020-10-22T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,163,9,172,...,359,400,8862,719,9714,3154,12868,478611,219012,NaN
4,18,2020-10-22T17:00:00,ITA,5,Veneto,45.434905,12.338452,500,59,559,...,1175,1325,24681,2301,23515,16075,39590,2192554,854703,NaN
5,5,2020-10-22T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,83,14,97,...,119,220,4482,368,6490,585,7075,492435,214444,NaN
6,7,2020-10-22T17:00:00,ITA,7,Liguria,44.411493,8.932699,540,34,574,...,445,690,13842,1673,14535,6046,20581,394365,207034,NaN
7,4,2020-10-22T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,636,86,722,...,863,889,27277,4537,35402,8075,43477,1433217,783681,NaN
8,15,2020-10-22T17:00:00,ITA,9,Toscana,43.769231,11.255889,524,82,606,...,1022,1145,12121,1229,20938,5673,26611,962277,646277,Sono stati effettuati 758 tamponi antigenici r...
9,16,2020-10-22T17:00:00,ITA,10,Umbria,43.106758,12.388247,171,22,193,...,315,407,2493,97,2240,3620,5860,266398,156285,NaN


In [10]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [11]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [12]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-23T17:00:00,1,Piemonte,1362,84,1446,15016,16462,1682,2032,31002,4236,27151,24549,51700,919629,547164,4341375
1,2020-10-23T17:00:00,2,Valle d'Aosta,50,3,53,948,1001,96,97,1166,149,2076,240,2316,37111,23862,125501
2,2020-10-23T17:00:00,3,Lombardia,2013,184,2197,35753,37950,4432,4916,88536,17159,119459,24186,143645,2647681,1655276,10103969
3,2020-10-23T17:00:00,4,Trentino-Alto Adige,171,11,182,3529,3711,424,476,8914,719,10098,3246,13344,484932,221317,1074819
4,2020-10-23T17:00:00,5,Veneto,515,64,579,13455,14034,1426,1550,24798,2308,23734,17406,41140,2208831,860733,4907704
5,2020-10-23T17:00:00,6,Friuli Venezia Giulia,88,19,107,2370,2477,252,340,4569,369,6792,623,7415,497736,216003,1211357
6,2020-10-23T17:00:00,7,Liguria,598,33,631,4838,5469,403,778,14210,1680,15062,6297,21359,399461,209220,1543127
7,2020-10-23T17:00:00,8,Emilia-Romagna,657,86,743,11771,12514,851,888,27304,4547,35818,8547,44365,1449195,790989,4467118
8,2020-10-23T17:00:00,9,Toscana,574,82,656,13718,14374,1113,1290,12290,1237,21997,5904,27901,973655,653719,3722729
9,2020-10-23T17:00:00,10,Umbria,183,21,204,3426,3630,360,447,2577,100,2287,4020,6307,269993,158186,880285


In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-23T17:00:00,1,Piemonte,1362,84,1446,15016,16462,1682,2032,31002,4236,27151,24549,51700,919629,547164,4341375,367
1,2020-10-23T17:00:00,2,Valle d'Aosta,50,3,53,948,1001,96,97,1166,149,2076,240,2316,37111,23862,125501,20
2,2020-10-23T17:00:00,3,Lombardia,2013,184,2197,35753,37950,4432,4916,88536,17159,119459,24186,143645,2647681,1655276,10103969,983
3,2020-10-23T17:00:00,4,Trentino-Alto Adige,171,11,182,3529,3711,424,476,8914,719,10098,3246,13344,484932,221317,1074819,106
4,2020-10-23T17:00:00,5,Veneto,515,64,579,13455,14034,1426,1550,24798,2308,23734,17406,41140,2208831,860733,4907704,825
5,2020-10-23T17:00:00,6,Friuli Venezia Giulia,88,19,107,2370,2477,252,340,4569,369,6792,623,7415,497736,216003,1211357,175
6,2020-10-23T17:00:00,7,Liguria,598,33,631,4838,5469,403,778,14210,1680,15062,6297,21359,399461,209220,1543127,209
7,2020-10-23T17:00:00,8,Emilia-Romagna,657,86,743,11771,12514,851,888,27304,4547,35818,8547,44365,1449195,790989,4467118,516
8,2020-10-23T17:00:00,9,Toscana,574,82,656,13718,14374,1113,1290,12290,1237,21997,5904,27901,973655,653719,3722729,415
9,2020-10-23T17:00:00,10,Umbria,183,21,204,3426,3630,360,447,2577,100,2287,4020,6307,269993,158186,880285,70


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-23T17:00:00,3,Lombardia,2013,184,2197,35753,37950,4432,4916,...,18.72,4076,791,36963,22167,7,477,13.30,0.04865,1.42
1,2020-10-23T17:00:00,15,Campania,1090,98,1188,23447,24635,2074,2280,...,22.95,2017,739,15801,12641,12,194,14.43,0.03941,0.59
2,2020-10-23T17:00:00,1,Piemonte,1362,84,1446,15016,16462,1682,2032,...,22.89,1541,482,12665,9702,9,341,16.04,0.04681,1.19
3,2020-10-23T17:00:00,5,Veneto,515,64,579,13455,14034,1426,1550,...,7.76,1406,225,16277,6030,7,117,9.52,0.03158,0.84
4,2020-10-23T17:00:00,12,Lazio,1354,135,1489,18332,19821,1290,1389,...,18.07,1225,138,22457,18457,11,88,6.19,0.02368,0.53
5,2020-10-23T17:00:00,9,Toscana,574,82,656,13718,14374,1113,1290,...,19.76,1063,145,11378,7442,8,169,11.34,0.03465,0.75
6,2020-10-23T17:00:00,8,Emilia-Romagna,657,86,743,11771,12514,851,888,...,16.67,830,-1,15978,7308,10,27,5.56,0.01988,0.99
7,2020-10-23T17:00:00,7,Liguria,598,33,631,4838,5469,403,778,...,15.79,346,88,5096,2186,7,368,15.27,0.05042,1.38
8,2020-10-23T17:00:00,19,Sicilia,593,89,682,8454,9136,596,730,...,16.54,591,-66,8015,5232,11,123,9.11,0.01469,0.31
9,2020-10-23T17:00:00,16,Puglia,479,52,531,6297,6828,549,590,...,14.21,516,105,5253,5008,1,40,11.23,0.01472,0.33


In [16]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)